# Demo 1: Building a Basic Chatbot with LLMs

In this demo, you'll learn how to interact with OpenAI's API to build a simple chatbot. We'll explore:

1. **Basic API calls** - Making your first request
2. **Conversation context** - Maintaining chat history
3. **System prompts** - Controlling bot behavior

## Learning Objectives

By the end of this demo, you'll understand:
- How to make basic API calls to LLMs
- How to build a stateful conversation
- How system prompts shape bot behavior

## Setup

First, let's import the required libraries and set up our API key.

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json

load_dotenv()

# Set your API key (get from environment or replace with your key)
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")  # Optional: for custom endpoints

print(api_key)

# Initialize the client
# For Vocareum keys, use: base_url="https://openai.vocareum.com/v1"
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

print("✅ OpenAI client initialized!")

voc-10694876731266774594167694c2af1d65632.81978761
✅ OpenAI client initialized!


## Part 1: Basic API Call

Let's start with a simple question and see how the LLM responds.

In [3]:
# Simple question
question = "What is an embedding?"

# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": question}
    ],
    temperature=0.7,
    max_tokens=100
)

# Extract the answer
answer = response.choices[0].message.content

print(f"Question: {question}")
print(f"\nAnswer: {answer}")

Question: What is an embedding?

Answer: An embedding is a method used in natural language processing and machine learning to represent words or phrases as vectors in a continuous vector space. This allows words to be represented numerically, making it easier for machine learning algorithms to process and analyze textual data. Embeddings capture semantic and syntactic relationships between words, enabling algorithms to understand the context and meaning of words in a given text. Popular embedding techniques include Word2Vec, GloVe, and FastText.


### 🤔 What Just Happened?

The LLM generated a response by **predicting the next most likely word** repeatedly. This simple mechanism enables complex, coherent responses!

## Part 2: Building a Stateful Conversation

LLMs are **stateless** - they don't remember previous messages unless you include them! Let's build a proper conversation.

In [4]:
# Initialize conversation history
conversation = []

def chat(user_message):
    """Send a message and get a response, maintaining conversation history."""
    
    # Add user message to history
    conversation.append({
        "role": "user",
        "content": user_message
    })
    
    # Make API call with full conversation history
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation,
        temperature=0.7,
        max_tokens=150
    )
    
    # Get assistant's response
    assistant_message = response.choices[0].message.content
    
    # Add assistant's response to history
    conversation.append({
        "role": "assistant",
        "content": assistant_message
    })
    
    return assistant_message

# Have a conversation!
print("User: What's the weather like today?")
response1 = chat("What's the weather like today?")
print(f"Bot: {response1}\n")

print("User: Should I bring an umbrella?")
response2 = chat("Should I bring an umbrella?")  # References previous context!
print(f"Bot: {response2}\n")

print("User: Thanks! What about tomorrow?")
response3 = chat("Thanks! What about tomorrow?")  # Still maintaining context
print(f"Bot: {response3}")

User: What's the weather like today?
Bot: I'm sorry, I am an AI assistant and do not have real-time information. Please check a weather website or app for the current weather in your location.

User: Should I bring an umbrella?
Bot: It is always a good idea to bring an umbrella with you just in case, especially if there is a chance of rain in the forecast. It's better to be prepared!

User: Thanks! What about tomorrow?
Bot: I apologize for the inconvenience, but as an AI assistant, I do not have the ability to provide real-time weather updates or forecasts. I recommend checking a reliable weather website or app for the most up-to-date information on tomorrow's weather in your location.


### 📝 View Conversation History

Let's see what we're sending to the API with each request:

In [5]:
print("Current conversation history:\n")
print(json.dumps(conversation, indent=2))

Current conversation history:

[
  {
    "role": "user",
    "content": "What's the weather like today?"
  },
  {
    "role": "assistant",
    "content": "I'm sorry, I am an AI assistant and do not have real-time information. Please check a weather website or app for the current weather in your location."
  },
  {
    "role": "user",
    "content": "Should I bring an umbrella?"
  },
  {
    "role": "assistant",
    "content": "It is always a good idea to bring an umbrella with you just in case, especially if there is a chance of rain in the forecast. It's better to be prepared!"
  },
  {
    "role": "user",
    "content": "Thanks! What about tomorrow?"
  },
  {
    "role": "assistant",
    "content": "I apologize for the inconvenience, but as an AI assistant, I do not have the ability to provide real-time weather updates or forecasts. I recommend checking a reliable weather website or app for the most up-to-date information on tomorrow's weather in your location."
  }
]


## Part 3: System Prompts - Controlling Behavior

System prompts are **secret instructions** that define how the bot should behave. The user never sees them, but they dramatically affect responses!

In [6]:
def create_bot_with_personality(system_prompt):
    """Create a bot with a specific personality."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Tell me about troubleshooting."}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7,
        max_tokens=100
    )
    
    return response.choices[0].message.content

# Test different personalities
print("🤖 Professional Bot:")
professional = create_bot_with_personality(
    "You are a professional tech support assistant. Be formal and concise."
)
print(professional)

print("\n" + "="*80 + "\n")

print("😊 Friendly Bot:")
friendly = create_bot_with_personality(
    "You are a friendly, enthusiastic tech support assistant. Use casual language and be upbeat!"
)
print(friendly)

print("\n" + "="*80 + "\n")

print("🎭 Pirate Bot (just for fun!):")
pirate = create_bot_with_personality(
    "You are a pirate tech support assistant. Respond in pirate speak with 'arr' and 'matey'!"
)
print(pirate)

🤖 Professional Bot:
Troubleshooting is the process of identifying, diagnosing, and resolving problems within a system or device. It involves systematically investigating the issue to determine its root cause and implementing a solution to fix the problem. Troubleshooting often requires analytical thinking, knowledge of the system or device, and the ability to follow a logical step-by-step approach to resolve technical issues effectively.


😊 Friendly Bot:
Hey there! Troubleshooting is basically trying to figure out why something isn't working the way it should and then fixing it. It's like being a detective for tech problems! Whether it's a glitchy app, a slow computer, or a wonky internet connection, troubleshooting helps you get things back on track. Just stay calm, take it step by step, and you'll usually find a solution! Got any tech issues you need help with?


🎭 Pirate Bot (just for fun!):
Arr, troubleshooting be the art of findin' and fixin' problems with yer devices, matey. It 

## Part 4: Building a Tech Support Bot

Now let's put it all together to build a simple tech support chatbot!

In [7]:
# Initialize a tech support bot
tech_support_bot = [
    {
        "role": "system",
        "content": """You are a helpful tech support assistant for TechCo, a software company.
        
Your responsibilities:
- Troubleshoot software issues and bugs
- Help with installation and setup problems
- Explain error messages
- Guide users through configuration steps

Guidelines:
- Be patient and clear in your explanations
- Ask diagnostic questions to identify the problem
- Provide step-by-step solutions
- If the issue requires developer attention, offer to create a support ticket
"""
    }
]

def tech_support_chat(user_message):
    """Tech support chat function."""
    tech_support_bot.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=tech_support_bot,
        temperature=0.7,
        max_tokens=200
    )
    
    assistant_message = response.choices[0].message.content
    tech_support_bot.append({"role": "assistant", "content": assistant_message})
    
    return assistant_message

# Simulate tech support interactions
test_messages = [
    "My application keeps crashing when I try to export a file.",
    "I'm using version 2.5 on Windows 11.",
    "I tried that but it's still not working. What else can I do?"
]

for msg in test_messages:
    print(f"👤 User: {msg}")
    response = tech_support_chat(msg)
    print(f"🤖 Bot: {response}\n")
    print("="*80 + "\n")

👤 User: My application keeps crashing when I try to export a file.
🤖 Bot: I'm sorry to hear that you're experiencing issues with exporting files in the application. Let's troubleshoot this together. 

First, could you provide more details about the crash? Do you receive any error messages when the application crashes? 

Also, have you tried exporting different types of files, or is it happening with a specific file format? 

Lastly, please let me know which operating system you're using so we can tailor the solution to your setup.


👤 User: I'm using version 2.5 on Windows 11.
🤖 Bot: Thank you for providing that information. Since you're using the application on Windows 11, let's try a few troubleshooting steps to address the crashing issue when exporting files:

1. **Check for Updates**: Make sure you are using the latest version of the application. Sometimes, updates include bug fixes that can resolve crashing issues.

2. **Check File Size**: If you are trying to export a very large 

## 🎯 Key Takeaways

1. **LLMs predict the next word** - that's the fundamental operation, but it enables complex behaviors

2. **Conversation requires state** - you must include message history in each API call

3. **System prompts are powerful** - they shape behavior without the user seeing them

4. **Structure matters** - messages have roles (system, user, assistant) that guide the model

## 💰 Cost Considerations

Each API call costs money based on tokens:
- System prompt: counted every time
- Conversation history: grows with each turn
- New message: adds more tokens
- Response: output tokens cost more!

**Pro tip**: Long conversations get expensive. Consider truncating old history in production.

## 🚀 Next Steps

Try these experiments:
1. Change the temperature (0 = deterministic, 1+ = creative)
2. Modify the system prompt to create different personalities
3. Add intent classification before generating responses
4. Implement conversation summarization for long chats

Move on to **Demo 2: Tokenization** to learn about cost optimization!